In [30]:
## Samuel Méndez Villegas - A016522777
## Uso de un framework/librería para implementar un modelo de ML

'''
En este código se presenta la implementación de un modelo de Machine Learning (ML), más concretamente de
un 'Random Forest Classifier'. Este modelo forma parte de la lista de algoritmos supervisados de ML y se caracterisa 
por crear 'n' subconjuntos de datos del set de entrenamiento y posteriormente, generar un árbol de decisión por cada 
subconjunto. Después de la creación de los árboles, se realiza una predicción de cada árbol, y se obtiene un resultado.
Dichos resultados pasan a un tipo de 'votación' en donde se escoge la clase más votada (en caso de clasificación), o 
el promedio de los resultados (en caso de regresión).

Para la implementación de esta librería, se utilizará la librería scikit-learn la cual incluye la función
'RandomForestClassifier()' la cual recibe el conjunto de entrenamiento y posteriormente el conjunto de prueba para
realizar predicciones.

En cuanto a la base de datos que se utilizará, es la base de datos de diabetes, en la cual se tratará de predecir
si un individuo padece de diabetes dada una serie de características como el nivel de glucosa, el número embarazos,
la edad, entre otras.

Finalmente, se realiza la evaluación del modelo implementado con ayuda de algunas métricas como el 'mean squared 
error', el 'mean absolute error', y la matriz de confusión. 
'''

## Librerías a utilizar
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn

## Librerías para implementar el modelo
from sklearn.model_selection import train_test_split # Separar el set de datos en entrenamiento y prueba
from sklearn.ensemble import RandomForestClassifier # Clasificador a utilizar
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, confusion_matrix, accuracy_score # Medidas de evaluación

In [2]:
'''
EXPLORACIÓN DE LA BASE DE DATOS Y UN POCO DE LIMPIEZA

En esta pequeña sección se pasará a visualizar los datos y realizar un pequeño análisis exploratorio de éstos antes
de aplicar el modelo de machine learning.
'''
## Carga de los datos a utilizar
df = pd.read_csv('diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
## Dimensiones del data frame
df.shape

(768, 9)

In [4]:
## Evaluación de datos duplicados
df.duplicated().sum()

0

In [5]:
## Evaluación de datos nulos
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [6]:
## Análisis descriptivo de las variables numéricas
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [ ]:
'''
Gracias al resumen de estadística descriptiva que ofrece `describe`, se observa que se tienen registros con
valores de 0 en algunas columnas como presión sanguinea, o en el ínidice de masa muscular, por lo que se pasa 
a identificar dichos registros
'''

In [15]:
df[df['BloodPressure'] == 0].head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
7,10,115,0,0,0,35.3,0.134,29,0
15,7,100,0,0,0,30.0,0.484,32,1
49,7,105,0,0,0,0.0,0.305,24,0
60,2,84,0,0,0,0.0,0.304,21,0
78,0,131,0,0,0,43.2,0.270,26,1


In [10]:
df[df['Insulin'] == 0].head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
5,5,116,74,0,0,25.6,0.201,30,0
7,10,115,0,0,0,35.3,0.134,29,0


In [11]:
df[df['BMI'] == 0].head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
9,8,125,96,0,0,0.0,0.232,54,1
49,7,105,0,0,0,0.0,0.305,24,0
60,2,84,0,0,0,0.0,0.304,21,0
81,2,74,0,0,0,0.0,0.102,22,0
145,0,102,75,23,0,0.0,0.572,21,0


In [ ]:
'''
Se puede observar que la muchos de los registros tienen valores de 0, por lo que puede que en realidad se traten de
valores faltantes que se llenaron con ceros. Por lo tanto, para tener un modelo un poco más limpio, se pasará a
eliminar estos registros. 
'''

In [21]:
df = df[(df['BloodPressure'] > 0) & (df['Glucose'] > 0) & (df['SkinThickness'] > 0) & (df['Insulin'] > 0) & (df['BMI'] > 0)]
df.shape

(392, 9)

In [23]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000
mean,3.301020,122.627551,70.663265,29.145408,156.056122,33.086224,0.523046,30.864796,0.331633
std,3.211424,30.860781,12.496092,10.516424,118.841690,7.027659,0.345488,10.200777,0.471401
min,0.000000,56.000000,24.000000,7.000000,14.000000,18.200000,0.085000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,21.000000,76.750000,28.400000,0.269750,23.000000,0.000000
50%,2.000000,119.000000,70.000000,29.000000,125.500000,33.200000,0.449500,27.000000,0.000000
75%,5.000000,143.000000,78.000000,37.000000,190.000000,37.100000,0.687000,36.000000,1.000000
max,17.000000,198.000000,110.000000,63.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [ ]:
'''
Como se puede observar en el nuevo describe, ya no se tienen valores mínimos en las variables en las no tenia sentido
tener un cero. Ya con este súper pequeño análisis de los datos, pasaremos a implementar el clasificador con bosques
aleatorios.
'''

In [24]:
## Selección de las variables predictoras y objetivo
x = df[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 
        'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']]
y = df['Outcome']

In [27]:
## Separación de la base de datos en un set de entrenamiento y prueba
X_train, x_test, Y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)
X_train.shape, x_test.shape, Y_train.shape, y_test.shape

((313, 8), (79, 8), (313,), (79,))

In [28]:
# Implementación del random forest classifier con los hiperparámetros default
rfc = RandomForestClassifier()
rfc.fit(X_train,Y_train) ## Entrenamiento del modelo

RandomForestClassifier()

In [29]:
## Se realizan las predicciones con el set de entrenamiento
train_results = rfc.predict(X_train)

In [31]:
## Se obtienen algunas medidas de evaluación como el mse, mae, la exactitud y la matriz de confusión
mse = mean_squared_error(train_results, Y_train)
mae = mean_absolute_error(train_results, Y_train)

print('Resultados de las predicciones con el set de entrenamiento:')
print('MSE con el conjunto de entrenamiento:', mse)
print('MAE con el conjunto de entrenamiento:', mae)
print("Accuracy con el conjunto de prueba:", accuracy_score(Y_train, train_results)) ## Indica que tanto el modelo clasificó correctamente
print(confusion_matrix(train_results, Y_train)) # Se muestra la matriz de confusión

MSE con el conjunto de entrenamiento: 0.0
MAE con el conjunto de entrenamiento: 0.0
Accuracy: 1.0
[[210   0]
 [  0 103]]


In [ ]:
'''
Como se puede observar en los resultados de la matriz de confusión y de algunas medidas como el accuracy, el mse y el
mae, el modelo es muy bueno para predecirse a si mismo, indicando de esta forma un posible overfitting.

Ahora se pasará a realizar las predicciones con el set de prueba.
'''

In [32]:
## Con los datos de prueba
test_results = rfc.predict(x_test)

In [35]:
## Se obtienen algunas medidas de evaluación como el mse, mae, la exactitud y la matriz de confusión
mse = mean_squared_error(test_results, y_test)
mae = mean_absolute_error(test_results, y_test)

print('Resultados de las predicciones con el set de prueba:')
print('MSE con el conjunto de prueba:', mse)
print('MAE con el conjunto de prueba:', mae)
print("Accuracy con el conjunto de prueba:", accuracy_score(y_test, test_results)) ## Indica que tanto el modelo clasificó correctamente
print(confusion_matrix(test_results, y_test)) # Se muestra la matriz de confusión

Resultados de las predicciones con el set de prueba:
MSE con el conjunto de prueba: 0.27848101265822783
MAE con el conjunto de prueba: 0.27848101265822783
Accuracy con el conjunto de prueba: 0.7215189873417721
[[45 15]
 [ 7 12]]


In [ ]:
'''
Se observa que la exactitud del modelo ya no es tan buena, pues al tener un overfitting, el modelo es malo para
predecir con nuevos registros. Sin embargo, si se habla del MSE y MAE se puede observar que estos no son tan grandes.

Ahora se pasará a implementar el mismo modelo de random forest, pero configurando los hiperparámetros con el objetivo
de mejorar al modelo y que éste realice mejores predicciones. 
'''

In [38]:
## Modificación de los hiperparámetros del modelo con el objetivo de mejorar el modelo
rfc = RandomForestClassifier(n_estimators = 1000, max_depth = 5, criterion = 'entropy', random_state = 42)
rfc.fit(X_train, Y_train) ## Entrenamiento del modelo

RandomForestClassifier(criterion='entropy', max_depth=5, n_estimators=1000,
                       random_state=42)

In [39]:
## Se realizan las predicciones en el conjunto de entrenamiento
train_results = rfc.predict(X_train)

In [40]:
## Se obtienen algunas de las medidas de evaluación
mse = mean_squared_error(train_results, Y_train)
mae = mean_absolute_error(train_results, Y_train)

print('Resultados de las predicciones con el set de entrenamiento:')
print('MSE con el conjunto de entrenamiento:', mse)
print('MAE con el conjunto de entrenamiento:', mae)
print("Accuracy con el conjunto de prueba:", accuracy_score(Y_train, train_results)) ## Indica que tanto el modelo clasificó correctamente
print(confusion_matrix(train_results, Y_train)) # Se muestra la matriz de confusión

Resultados de las predicciones con el set de entrenamiento:
MSE con el conjunto de entrenamiento: 0.0926517571884984
MAE con el conjunto de entrenamiento: 0.0926517571884984
Accuracy con el conjunto de prueba: 0.9073482428115016
[[201  20]
 [  9  83]]


In [ ]:
'''
Desde aquí se observa que este modelo mejoró algo en consideración al anterior, en el sentido de que ya no se 
tiene un sobre ajuste de los datos, pues como se observa el modelo tiende fallar, poco, pero falla. Ahora probaremos
su rendimiento con el conjunto de prueba.
'''

In [41]:
## Con los datos de prueba
test_results = rfc.predict(x_test)

In [42]:
mse = mean_squared_error(test_results, y_test)
mae = mean_absolute_error(test_results, y_test)
r2 = r2_score(test_results, y_test)

print('Resultados de las predicciones con el set de prueba:')
print('MSE con el conjunto de prueba:', mse)
print('MAE con el conjunto de prueba:', mae)
print("Accuracy con el conjunto de prueba:", accuracy_score(y_test, test_results)) ## Indica que tanto el modelo clasificó correctamente
print(confusion_matrix(test_results, y_test)) # Se muestra la matriz de confusión

Resultados de las predicciones con el set de prueba:
MSE con el conjunto de prueba: 0.25316455696202533
MAE con el conjunto de prueba: 0.25316455696202533
Accuracy con el conjunto de prueba: 0.7468354430379747
[[45 13]
 [ 7 14]]


In [ ]:
'''
Finalmente, se observa que en realidad el modelo tuvo una mejora, sin embargo esta no fue del todo buena, ya que
solamente aumentó en 2% y de igual forma tanto el mae como el mse están más cercanos a 0. En la siguiente actividad
se realizará un análisis mucho más profundo de la evaluación del modelo con el objetivo de mejorar este mucho más.
'''